# Score: 99.142%

### Imports

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

### Data

In [ ]:
train = pd.read_csv("/Users/pbezuhov/git/Kaggle/data/mnist/train.csv")
test = pd.read_csv("/Users/pbezuhov/git/Kaggle/data/mnist/test.csv")

### Processing the data

In [2]:
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

train_y = train['label']
train_x = train.drop(['label'], axis=1)

# Reshape
train_x = train_x.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# Normalize
train_x = train_x / 255.0
test = test / 255.0

train_y = to_categorical(train_y, num_classes = 10)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.2)

Using TensorFlow backend.


### Creating the model

In [ ]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Flatten

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5), 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(filters = 64, kernel_size = (3,3),
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.4))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

# Compile the model
model.compile(optimizer="adam", loss = "categorical_crossentropy", metrics=["accuracy"])

### Generating more images

In [4]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1  # randomly shift images vertically (fraction of total height)
)


data_generator.fit(train_x)

# Fitting the model to the data

### Low batch size

In [7]:
epochs = 10
batch_size = 30

history = model.fit_generator(data_generator.flow(train_x, train_y, batch_size=batch_size),
                              epochs = epochs, validation_data = (val_x, val_y))

Epoch 1/10
1120/1120 [==============================] - 183s 163ms/step - loss: 0.1877 - acc: 0.9471 - val_loss: 0.0531 - val_acc: 0.9861
Epoch 2/10
1120/1120 [==============================] - 208s 186ms/step - loss: 0.1367 - acc: 0.9624 - val_loss: 0.0488 - val_acc: 0.9870
Epoch 3/10
1120/1120 [==============================] - 234s 209ms/step - loss: 0.1201 - acc: 0.9680 - val_loss: 0.0441 - val_acc: 0.9888
Epoch 4/10
1120/1120 [==============================] - 188s 168ms/step - loss: 0.1100 - acc: 0.9709 - val_loss: 0.0449 - val_acc: 0.9887
Epoch 5/10
1120/1120 [==============================] - 201s 180ms/step - loss: 0.0938 - acc: 0.9749 - val_loss: 0.0389 - val_acc: 0.9898
Epoch 6/10
1120/1120 [==============================] - 190s 170ms/step - loss: 0.0923 - acc: 0.9755 - val_loss: 0.0363 - val_acc: 0.9917
Epoch 7/10
1120/1120 [==============================] - 202s 180ms/step - loss: 0.0845 - acc: 0.9776 - val_loss: 0.0309 - val_acc: 0.9926
Epoch 8/10
1120/1120 [============

# Predictions

In [11]:
predictions = model.predict(test)

submission = pd.DataFrame({
                "ImageId": range(1,28001),
                "Label": np.argmax(predictions, axis = 1)
                })

# Saving the predictions

In [10]:
submission.to_csv("/Users/pbezuhov/git/Kaggle/submissions/3_small_batch_size.csv", index=False)